# **KAYAK - RECOMMANDATIONS DE DESTINATIONS EN FRANCE**
### **BLOC 01 : CONSTRUIRE ET GÉRER UNE INFRASTRUCTURE DE DONNÉES**

---

## 1. Initialisation et importations

In [1]:
import pandas as pd
import requests
import json
import time
from datetime import datetime
import plotly.express as px
import boto3
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
import datetime

In [171]:
# Liste des villes françaises à traiter

french_cities = [
    "Mont-Saint-Michel", "Saint-Malo", "Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille",
    "Strasbourg", "Haut-Koenigsbourg", "Colmar", "Eguisheim", "Besancon", "Dijon",
    "Annecy", "Grenoble", "Lyon", "Gorges du Verdon", "Bormes-les-Mimosas", "Cassis",
    "Marseille", "Aix-en-Provence", "Avignon", "Uzès", "Nîmes", "Aigues-Mortes",
    "Saintes-Maries-de-la-Mer", "Collioure", "Carcassonne", "Ariège", "Toulouse",
    "Montauban", "Biarritz", "Bayonne", "La Rochelle"
]

print(f"Nombre de villes à traiter : {len(french_cities)}")

Nombre de villes à traiter : 35


## 2. Collecte des données météo

### 2.1. Obtenir les coordonnées GPS des villes (Nominatim)

In [162]:
# Création d'un DataFrame pour stocker les informations des villes

cities_df = pd.DataFrame(french_cities, columns=['city_name'])
cities_df['city_id'] = cities_df.index
cities_df['latitude'] = None
cities_df['longitude'] = None

print("Récupération des coordonnées GPS pour chaque ville...")
for index, row in cities_df.iterrows():
    city = row['city_name']
    
    url = f"https://nominatim.openstreetmap.org/search?q={city},France&format=json&limit=1"
    
    headers = {
        'User-Agent': 'sterenngeleoc@gmail.com'
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
        if data:
            cities_df.loc[index, 'latitude'] = float(data[0]['lat'])
            cities_df.loc[index, 'longitude'] = float(data[0]['lon'])
        else:
            print(f"Aucune coordonnée trouvée pour {city}")
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la requête Nominatim pour {city}: {e}")
        
    time.sleep(1) # Respecter les limites de requêtes de Nominatim (1 req/sec)

print("\nCoordonnées GPS récupérées :")
print(cities_df.head())

Récupération des coordonnées GPS pour chaque ville...

Coordonnées GPS récupérées :
           city_name  city_id   latitude longitude
0  Mont-Saint-Michel        0  48.635954  -1.51146
1         Saint-Malo        1  48.649518 -2.026041
2             Bayeux        2  49.276462 -0.702474
3           Le Havre        3  49.493898  0.107973
4              Rouen        4  49.440459  1.093966


In [163]:
# Sauvegarde des données dans un fichier CSV
csv_filename = 'french_cities_coordinates.csv'
cities_df.to_csv(csv_filename, index=False)
print(f"\nDonnées sauvegardées dans {csv_filename}")


Données sauvegardées dans french_cities_coordinates.csv


In [2]:
cities_df = pd.read_csv('french_cities_coordinates.csv')

### 2.2. Obtenir les données météo (OpenWeatherMap)

In [2]:
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")

In [ ]:
# DataFrame pour stocker les données météo détaillées

weather_data = []

print("\nRécupération des données météo pour chaque ville (prévisions sur 7 jours)...")
for index, row in cities_df.iterrows():
    city_id = row['city_id'] 
    city_name = row['city_name']
    lat = row['latitude']
    lon = row['longitude']
    
    try:
        response = requests.get("https://api.openweathermap.org/data/3.0/onecall",
                     params={"lat":lat,"lon":lon,"appid":key,"units":"metric","lang":"fr"})
        response.raise_for_status() # Lèvera toujours une erreur 401 si la clé n'est pas bonne
        data = response.json()

        for day_data in data.get('daily', []):
            weather_data.append({
                'city_id': city_id,
                'city_name': city_name,
                'dt': datetime.datetime.fromtimestamp(day_data['dt']).strftime('%Y-%m-%d'), # Date de la prévision
                'temp_day': day_data['temp']['day'], # Température moyenne de la journée
                'humidity': day_data['humidity'], # Humidité en %
                'wind' : day_data['wind_speed'], # Vitesse du vent en m/s
                'weather_main': day_data['weather'][0]['main'], # Description principale du temps
                'rain' : day_data.get('rain', 0), # Précipitations en mm (0 si non présent)
                'pop': day_data['pop'], # Probabilité de précipitation
            })
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la requête OpenWeatherMap pour {city_name}: {e}")
    time.sleep(1) 

weather_df = pd.DataFrame(weather_data)
print("\nDonnées météo récupérées :")
print(weather_df.head())


Récupération des données météo pour chaque ville (prévisions sur 7 jours)...

Données météo récupérées :
   city_id          city_name          dt  temp_day  humidity  wind  \
0        0  Mont-Saint-Michel  2025-10-09     17.06        67  5.81   
1        0  Mont-Saint-Michel  2025-10-10     17.69        59  4.43   
2        0  Mont-Saint-Michel  2025-10-11     19.47        62  4.69   
3        0  Mont-Saint-Michel  2025-10-12     19.69        53  5.21   
4        0  Mont-Saint-Michel  2025-10-13     21.02        53  5.90   

  weather_main  rain  pop  
0       Clouds   0.0  0.0  
1       Clouds   0.0  0.0  
2       Clouds   0.0  0.0  
3       Clouds   0.0  0.0  
4        Clear   0.0  0.0  


In [106]:
# Sauvegarde des données dans un fichier CSV
csv_weather_filename = 'french_cities_weather.csv'
weather_df.to_csv(csv_weather_filename, index=False)
print(f"\nDonnées météo sauvegardées dans {csv_weather_filename}")


Données météo sauvegardées dans french_cities_weather.csv


In [3]:
weather_df = pd.read_csv('french_cities_weather.csv')

### 2.3. Déterminer les meilleures destinations

Définition des critères pour une "belle météo" :
- Température moyenne agréable (entre 18 et 25 degrés Celsius)
- Faible probabilité de précipitation (pop < 0.2)
- Peu ou pas de pluie (rain == 0)
- Faible humidité (humidity < 80)
- Vitesse du vent modérée (wind < 10 m/s)

In [176]:
# Calcul des scores pour chaque ville sur les 7 prochains jours
# Agrégation des données météo par ville pour les 7 jours

weather_summary = weather_df.groupby('city_id').agg(
    avg_temp_day=('temp_day', 'mean'),
    total_rain=('rain', 'sum'),
    avg_pop=('pop', 'mean'),
    avg_humidity=('humidity', 'mean'),
    avg_wind_speed=('wind', 'mean'),
    clear_sky_days=('weather_main', lambda x: (x == 'Clear').sum()),
    cloudy_days=('weather_main', lambda x: (x == 'Clouds').sum()),
    rainy_days=('weather_main', lambda x: (x == 'Rain').sum()),
).reset_index()

In [ ]:
# Calcul d'un score de "beau temps" (plus le score est élevé, plus la météo est considérée comme agréable)
weather_summary['weather_score'] = 0

# Température idéale (entre 18 et 25 degrés)
weather_summary['weather_score'] += weather_summary.apply(
    lambda x: 10 if 18 <= x['avg_temp_day'] <= 25 else (5 if 10 <= x['avg_temp_day'] < 18 or 25 < x['avg_temp_day'] <= 30 else 0), axis=1
)

# Faible probabilité de précipitation
weather_summary['weather_score'] += weather_summary.apply(
    lambda x: 10 if x['avg_pop'] < 0.2 else (5 if x['avg_pop'] < 0.5 else 0), axis=1
)

# Peu ou pas de pluie
weather_summary['weather_score'] += weather_summary.apply(
    lambda x: 10 if x['total_rain'] == 0 else (5 if x['total_rain'] < 5 else 0), axis=1
)

# Faible humidité
weather_summary['weather_score'] += weather_summary.apply(
    lambda x: 5 if x['avg_humidity'] < 70 else (2 if x['avg_humidity'] < 85 else 0), axis=1
)

# Vitesse du vent modérée
weather_summary['weather_score'] += weather_summary.apply(
    lambda x: 5 if x['avg_wind_speed'] < 5 else (2 if x['avg_wind_speed'] < 10 else 0), axis=1
)

# Jours ensoleillés
weather_summary['weather_score'] += weather_summary['clear_sky_days'] * 2

In [186]:
# Fusion avec le DataFrame principal des villes

weather_summary = weather_summary.merge(cities_df, on='city_id', how='left')
weather_summary

,city_id,avg_temp_day,total_rain,avg_pop,avg_humidity,avg_wind_speed,clear_sky_days,cloudy_days,rainy_days,weather_score,city_name,latitude,longitude
0,0,18.55875,0.00,0.00000,56.875,5.58250,3,5,0,43,Mont-Saint-Michel,48.635954,-1.511460
1,1,17.02500,0.00,0.00000,66.875,7.35750,2,6,0,36,Saint-Malo,48.649518,-2.026041
2,2,16.69250,0.00,0.00375,65.500,6.30250,1,7,0,34,Bayeux,49.276462,-0.702474
3,3,16.76875,0.36,0.12500,66.875,6.68625,1,6,1,29,Le Havre,49.493898,0.107973
4,4,17.89250,0.00,0.00000,58.375,5.00750,2,6,0,36,Rouen,49.440459,1.093966
5,5,17.68250,0.00,0.00000,53.000,3.86875,4,4,0,43,Paris,48.853495,2.348391
6,6,17.03125,0.00,0.00000,60.875,4.80500,0,8,0,35,Amiens,49.894171,2.295695
7,7,16.57625,0.21,0.02500,64.375,3.99875,2,5,1,34,Lille,50.636565,3.063528
8,8,16.47125,0.00,0.00000,63.375,3.69250,4,4,0,43,Strasbourg,48.584614,7.750713
9,9,14.32375,0.00,0.00000,63.375,2.64375,4,4,0,43,Haut-Koenigsbourg,48.249411,7.344320


### 2.4. Sauvegarder les données météo agrégées en CSV

In [189]:
# Sauvegarde du DataFrame complet des villes avec les données météo agrégées

weather_summary.to_csv('french_cities_weather_data.csv', index=False)

print("\nDonnées météo agrégées pour les villes sauvegardées dans 'french_cities_weather_data.csv'")


Données météo agrégées pour les villes sauvegardées dans 'french_cities_weather_data.csv'


In [79]:
weather_cities = pd.read_csv('french_cities_weather_data.csv')
print(weather_cities.head())
weather_cities.info()

   city_id  avg_temp_day  total_rain  avg_pop  avg_humidity  avg_wind_speed  \
0        0      18.55875        0.00  0.00000        56.875         5.58250   
1        1      17.02500        0.00  0.00000        66.875         7.35750   
2        2      16.69250        0.00  0.00375        65.500         6.30250   
3        3      16.76875        0.36  0.12500        66.875         6.68625   
4        4      17.89250        0.00  0.00000        58.375         5.00750   

   clear_sky_days  cloudy_days  rainy_days  weather_score          city_name  \
0               3            5           0             43  Mont-Saint-Michel   
1               2            6           0             36         Saint-Malo   
2               1            7           0             34             Bayeux   
3               1            6           1             29           Le Havre   
4               2            6           0             36              Rouen   

    latitude  longitude  
0  48.635954  -1.5

## 3. Scraping des hôtels (Booking.com)

***Voir le dossier 'booking_scraper'***

In [81]:
# Chargement des données d'hôtels scrappées

hotels_df = pd.read_csv('hotels.csv')

print(hotels_df.info())
hotels_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 875 entries, 0 to 874
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   city         875 non-null    object 
 1   description  875 non-null    object 
 2   latitude     874 non-null    float64
 3   longitude    874 non-null    float64
 4   name         875 non-null    object 
 5   score        873 non-null    object 
 6   url          875 non-null    object 
dtypes: float64(2), object(5)
memory usage: 48.0+ KB
None


,city,description,latitude,longitude,name,score,url
0,Mont-Saint-Michel,Hébergement géré par un particulier,48.615795,-1.488712,L'Hirondelle,"Avec une note de 8,7",https://www.booking.com/hotel/fr/l-39-hirondel...
1,Saint-Malo,"Situé côté océan, le Grand Hôtel Des Thermes e...",48.657838,-1.999345,Grand Hôtel Des Thermes,"Avec une note de 8,6",https://www.booking.com/hotel/fr/desthermessai...
2,Saint-Malo,"Doté d’une connexion Wi-Fi gratuite, l’héberge...",48.647761,-2.024890,Le Nuage Malouin - Appt à 300m de la plage,"Avec une note de 9,2",https://www.booking.com/hotel/fr/le-nuage-malo...
3,Saint-Malo,Hébergement géré par un particulier,48.655380,-2.003277,Studio cocooning,"Avec une note de 9,0",https://www.booking.com/hotel/fr/studio-cocoon...
4,Saint-Malo,L’établissement L'AccrocheCoeur bénéficie d’un...,48.648121,-2.026979,L'AccrocheCoeur,"Avec une note de 9,2",https://www.booking.com/hotel/fr/l-39-accroche...


In [ ]:
# Nettoyage de la colonne 'score'

hotels_df['score'] = hotels_df['score'].str.replace('Avec une note de ', '', regex=False)
hotels_df['score'] = hotels_df['score'].str.replace(',', '.', regex=False)
hotels_df['score'] = pd.to_numeric(hotels_df['score'], errors='coerce')

In [85]:
# Filtrer les lignes où latitude ou longitude est manquante
missing_lat_lon = hotels_df[hotels_df[['latitude', 'longitude']].isnull().any(axis=1)]
print(missing_lat_lon['name'])

170    Grand Hotel Bellevue - Grand Place
Name: name, dtype: object


In [ ]:
# Imputation dans le DataFrame
# L'API Nominatim donne pour le Grand Hôtel Bellevue, situé au 5 Rue Jean Roisin, 59800 Lille, 
# les coordonnées suivantes : Latitude : ≈50.63660 / Longitude : ≈3.06310

hotels_df.loc[hotels_df['name'] == 'Grand Hotel Bellevue - Grand Place', ['latitude', 'longitude']] = [50.63660, 3.06310]

## 4. Fusion des dataframes

In [87]:
df_final = pd.merge(
    hotels_df,
    weather_cities.drop(columns=['latitude', 'longitude']), # Évite la duplication des colonnes lat/lon (déjà dans hotels_df)
    left_on='city',
    right_on='city_name',
    how='left' # Conserve tous les hôtels
)
# Retirer la colonne 'city_name' du df fusionné car elle est redondante avec 'city'
df_final = df_final.drop(columns=['city_name'])

In [88]:
df_final.info()
df_final.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 875 entries, 0 to 874
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   city            875 non-null    object 
 1   description     875 non-null    object 
 2   latitude        875 non-null    float64
 3   longitude       875 non-null    float64
 4   name            875 non-null    object 
 5   score           873 non-null    float64
 6   url             875 non-null    object 
 7   city_id         875 non-null    int64  
 8   avg_temp_day    875 non-null    float64
 9   total_rain      875 non-null    float64
 10  avg_pop         875 non-null    float64
 11  avg_humidity    875 non-null    float64
 12  avg_wind_speed  875 non-null    float64
 13  clear_sky_days  875 non-null    int64  
 14  cloudy_days     875 non-null    int64  
 15  rainy_days      875 non-null    int64  
 16  weather_score   875 non-null    int64  
dtypes: float64(8), int64(5), object(4)


,city,description,latitude,longitude,name,score,url,city_id,avg_temp_day,total_rain,avg_pop,avg_humidity,avg_wind_speed,clear_sky_days,cloudy_days,rainy_days,weather_score
0,Mont-Saint-Michel,Hébergement géré par un particulier,48.615795,-1.488712,L'Hirondelle,8.7,https://www.booking.com/hotel/fr/l-39-hirondel...,0,18.55875,0.0,0.0,56.875,5.5825,3,5,0,43
1,Saint-Malo,"Situé côté océan, le Grand Hôtel Des Thermes e...",48.657838,-1.999345,Grand Hôtel Des Thermes,8.6,https://www.booking.com/hotel/fr/desthermessai...,1,17.02500,0.0,0.0,66.875,7.3575,2,6,0,36
2,Saint-Malo,"Doté d’une connexion Wi-Fi gratuite, l’héberge...",48.647761,-2.024890,Le Nuage Malouin - Appt à 300m de la plage,9.2,https://www.booking.com/hotel/fr/le-nuage-malo...,1,17.02500,0.0,0.0,66.875,7.3575,2,6,0,36
3,Saint-Malo,Hébergement géré par un particulier,48.655380,-2.003277,Studio cocooning,9.0,https://www.booking.com/hotel/fr/studio-cocoon...,1,17.02500,0.0,0.0,66.875,7.3575,2,6,0,36
4,Saint-Malo,L’établissement L'AccrocheCoeur bénéficie d’un...,48.648121,-2.026979,L'AccrocheCoeur,9.2,https://www.booking.com/hotel/fr/l-39-accroche...,1,17.02500,0.0,0.0,66.875,7.3575,2,6,0,36


In [44]:
df_final.to_csv('french_hotels_weather_final.csv', index=False)

## 4. Constitution du Data Lake (AWS S3)

Configuration d'AWS CLI (localement)

Configurer l'utilisateur IAM `kayak-etl-user` en exécutant dans le terminal :

`aws configure`

- Entrer l'Access Key ID et la Secret Access Key sauvegardées.
- Définir la région (eu-west-3 pour Paris).
- Définir le format de sortie (json).

### 4.1. Configuration et téléchargement vers AWS S3

In [ ]:
import boto3
import logging

BUCKET_NAME = 'kayak-sterenn'
FILE_NAME = 'french_hotels_weather_final.csv'
OBJECT_NAME = 'raw_data/' + FILE_NAME

# Initialisation du client S3
# boto3 utilise automatiquement les identifiants configurés avec 'aws configure'

try:
    s3_client = boto3.client('s3')
    s3_client.upload_file(FILE_NAME, BUCKET_NAME, OBJECT_NAME)
    print(f"Fichier {FILE_NAME} chargé avec succès vers s3://{BUCKET_NAME}/{OBJECT_NAME}")

except Exception as e:
    logging.error(e)
    print(f"Échec de l'upload S3: {e}")

Fichier french_hotels_weather_final.csv chargé avec succès vers s3://kayak-sterenn/raw_data/french_hotels_weather_final.csv


## 5. ETL (Extraction, Transformation, Chargement) vers un Data Warehouse (AWS RDS)

In [ ]:
df_kayak = pd.read_csv('french_hotels_weather_final.csv')

DB_TYPE = 'postgresql'
DB_DRIVER = 'psycopg2'
DB_USER = 'kayak_user'
DB_PASSWORD = os.getenv("DB_PASSWORD") 
DB_HOST = os.getenv("RDS_DB_HOST")   
DB_PORT = '5432'
DB_NAME = 'postgres' 

# Construire la chaîne de connexion
CONN_STRING = f"{DB_TYPE}+{DB_DRIVER}://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Créer l'objet moteur SQLAlchemy
engine = create_engine(CONN_STRING)

# Charger le DataFrame dans la DB
df_kayak.to_sql(
    'kayak_destinations',
    engine,
    if_exists='replace',
    index=False,
)

print(f"Données chargées avec succès dans la table 'kayak_destinations' sur RDS.")

Données chargées avec succès dans la table 'kayak_destinations' sur RDS.


## 6. Visualisations finales

In [71]:
df_kayak = pd.read_csv('french_hotels_weather_final.csv')
df_weather_cities = pd.read_csv('french_cities_weather_data.csv')

### 6.1. Visualisation des meilleures destinations (villes) selon la météo à 7 jours

***Les Scores Météo ont été calculés mi-octobre.***

In [72]:
# Création d'une colonne pour le texte du survol (sans le score météo pour éviter la duplication)
# Création d'une liste de données pour hover_data et utiliser un template personnalisé.
df_weather_cities['Température moyenne'] = df_weather_cities['avg_temp_day'].round(1).astype(str) + '°C'
df_weather_cities['Pluie totale (7j)'] = df_weather_cities['total_rain'].round(1).astype(str) + 'mm'
df_weather_cities['Score Météo'] = df_weather_cities['weather_score'].round(1)


# Carte de toutes les villes avec leur score météo (Utilisation de hover_data)
fig_all_cities = px.scatter_mapbox(df_weather_cities,
                                     lat="latitude",
                                     lon="longitude",
                                     color="Score Météo",  # Utilisation de la colonne renommée pour une meilleure légende
                                     size="Score Météo",
                                     hover_data={
                                         'latitude': False,
                                         'longitude': False,
                                         'city_name': True, # Le nom de la ville sera le titre de la bulle
                                         'Score Météo': True, # Le score météo doit rester pour la couleur/taille
                                         'Température moyenne': True,
                                         'Pluie totale (7j)': True
                                     },
                                     color_continuous_scale=px.colors.sequential.Viridis,
                                     zoom=5,
                                     height=800,
                                     title="Scores Météo des villes françaises (7 jours)",
                                     mapbox_style="open-street-map")

# Personnalisation finale de l'info-bulle (pour retirer les noms de colonnes indésirables)
fig_all_cities.update_traces(
    hovertemplate='<b>%{customdata[2]}</b><br><br>' + # Nom de la ville
                  'Score Météo: %{customdata[3]:.1f}<br>' + # Score Météo
                  'Température moyenne: %{customdata[0]}<br>' + # Température
                  'Pluie totale (7j): %{customdata[1]}<br>' + # Pluie
                  '<extra></extra>', # Retire le nom de la trace "trace 0"
    customdata=df_weather_cities[['Température moyenne', 'Pluie totale (7j)', 'city_name', 'Score Météo']]
)

fig_all_cities.show()

![Scores Météo des villes françaises (7 jours)](viz\01-meteo_scores_french_cities.png)

![meteo scores](viz\meteo_scores.png)

In [ ]:
#  Top 5 des villes par score météo
top5_weather_cities = df_weather_cities.sort_values(by='weather_score', ascending=False).head(5)

print("\nTop 5 des destinations avec la meilleure météo :")
print(top5_weather_cities[['city_name', 'avg_temp_day', 'total_rain', 'avg_pop', 'weather_score']].round(2))


Top 5 des destinations avec la meilleure météo :
      city_name  avg_temp_day  total_rain  avg_pop  weather_score
15     Grenoble         21.72         0.0     0.00             52
34  La Rochelle         19.41         0.0     0.00             51
20    Marseille         21.45         0.0     0.00             50
19       Cassis         20.72         0.0     0.00             50
22      Avignon         23.07         0.0     0.02             49


In [ ]:
# Carte du Top 5 des villes avec leur score météo (utilisation de hover_data)

fig_top5_cities = px.scatter_mapbox(top5_weather_cities,
                                     lat="latitude",
                                     lon="longitude",
                                     color="Score Météo",  # Utilisation de la colonne renommée pour une meilleure légende
                                     size="Score Météo",
                                     hover_data={
                                         'latitude': False,
                                         'longitude': False,
                                         'city_name': True, # Le nom de la ville sera le titre de la bulle
                                         'Score Météo': True, # Le score météo doit rester pour la couleur/taille
                                         'Température moyenne': True,
                                         'Pluie totale (7j)': True
                                     },
                                     color_continuous_scale=px.colors.sequential.Viridis,
                                     zoom=5,
                                     height=600,
                                     title="Top 5 des Scores Météo des villes françaises (7 jours)",
                                     mapbox_style="open-street-map")

# Personnalisation finale de l'info-bulle (pour retirer les noms de colonnes indésirables)
fig_top5_cities.update_traces(
    hovertemplate='<b>%{customdata[2]}</b><br><br>' + # Nom de la ville
                  'Score Météo: %{customdata[3]:.1f}<br>' + # Score Météo
                  'Température moyenne: %{customdata[0]}<br>' + # Température
                  'Pluie totale (7j): %{customdata[1]}<br>' + # Pluie
                  '<extra></extra>', # Retire le nom de la trace "trace 0"
    customdata=top5_weather_cities[['Température moyenne', 'Pluie totale (7j)', 'city_name', 'Score Météo']]
)

fig_top5_cities.show()

![Top 5 des Scores Météo des villes françaises (7 jours)](viz\02-top5_meteo_scores.png)

### 6.2. Visualisation des 20 meilleurs hébergements (score Booking.com) dans la zone (area) des 5 meilleures villes pour la météo (7 jours)

In [93]:
# Création d'un DataFrame filtré pour les hôtels situés dans la zone (area)
target_cities = ["Grenoble", "La Rochelle", "Marseille", "Cassis", "Avignon", "Aix-en-Provence", "Uzès", "Nîmes", 
                 "Saintes-Maries-de-la-Mer", "Lyon", "Annecy", "Gorges du Verdon", "Bormes-les-Mimosas", "Aigues-Mortes"]
df_area = df_kayak[df_kayak['city'].isin(target_cities)]

# Création du Top 20 des hôtels par score Booking.com dans cette zone
top20_hotels = df_area.sort_values(by='score', ascending=False).head(20)

In [91]:
# Créer une nouvelle colonne pour la taille des bulles
min_score = top20_hotels['score'].min()
top20_hotels['score_boosted'] = (top20_hotels['score'] - min_score) * 10 + 1

titre = "Top 20 des meilleurs hébergements (Score Booking.com) dans la zone des 5 meilleures villes pour la météo (7 jours)"

fig_top20_hotels = px.scatter_mapbox(top20_hotels,
                                     lat="latitude",
                                     lon="longitude",
                                     color="weather_score",
                                     size="score_boosted", # Variable transformée
                                     size_max=35,
                                     hover_name="name", # Le nom de l'hôtel sera le titre en gras
                                     #color_continuous_scale=px.colors.sequential.Plasma,
                                     color_continuous_scale=px.colors.sequential.Viridis,
                                     zoom=6,
                                     height=800,
                                     title=titre,
                                     mapbox_style="open-street-map")

# Personnalisation de l'info-bulle (hovertemplate)
# Indices de custom_data :
# city -> [0]
# score -> [1]
# weather_score -> [2]

custom_cols = ['city', 'score', 'weather_score', 'description'] # Nouvelle liste
custom_data = top20_hotels[custom_cols].values

fig_top20_hotels.update_traces(
    # %{hovertext} est 'name' (Nom de l'hôtel)
    hovertemplate='<b>%{hovertext}</b><br>' + # Nom de l'hôtel
                  'Ville: %{customdata[0]}<br>' + # Ville
                  'Score Booking: %{customdata[1]:.2f}<br>' + # Score
                  'Score Météo: %{customdata[2]}<br>' + # Score Météo
                  '<extra></extra>', # Supprime la ligne 'trace 0'
    customdata=custom_data
)

fig_top20_hotels.show()

![Top 20 des meilleurs hébergements (Score Booking.com) dans la zone des 5 meilleures villes pour la météo (7 jours)](viz\01-meteo_scores_french_cities.png)

**N.B. : plus le rond est grand, plus la note de Booking.com est élevée.**